In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import cv2
import json
import os
import matplotlib.pyplot as plt
import random
import seaborn as sns
from keras.models import Sequential
from keras import optimizers
from keras import backend as K
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [2]:
directory = r"C:\Users\arpit\Desktop\SAP-internal-face-mask-detection\Medical mask\Medical mask\Medical Mask\annotations"
image_directory = r"C:\Users\arpit\Desktop\SAP-internal-face-mask-detection\Medical mask\Medical mask\Medical Mask\images"
df = pd.read_csv(r"C:\Users\arpit\Desktop\SAP-internal-face-mask-detection\Medical mask\train.csv")
df_test = pd.read_csv(r"C:\Users\arpit\Desktop\SAP-internal-face-mask-detection\Medical mask\submission.csv")

In [3]:

cvNet = cv2.dnn.readNetFromCaffe(prototxt="models/deploy.prototxt",
                                            caffeModel="models/res10_300x300_ssd_iter_140000_fp16.caffemodel")
def getJSON(filePathandName):
    with open(filePathandName,'r') as f:
        return json.load(f)
def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)])
    return cv2.LUT(image.astype(np.uint8), table.astype(np.uint8))

In [9]:
df.head()

,name,x1,x2,y1,y2,classname
0,2756.png,69,126,294,392,face_with_mask
1,2756.png,505,10,723,283,face_with_mask
2,2756.png,75,252,264,390,mask_colorful
3,2756.png,521,136,711,277,mask_colorful
4,6098.jpg,360,85,728,653,face_no_mask


In [4]:
jsonfiles= []
for i in os.listdir(directory):
    jsonfiles.append(getJSON(os.path.join(directory,i)))
jsonfiles[0]

{'FileName': '1801.jpg',
 'NumOfAnno': 1,
 'Annotations': [{'isProtected': False,
   'ID': 924868908868875136,
   'BoundingBox': [451, 186, 895, 697],
   'classname': 'face_no_mask',
   'Confidence': 1,
   'Attributes': {}}]}

In [11]:
jsonfiles = []
for i in df['name'].unique():
    f = i + ".json"
    jsonfiles.append(getJSON(os.path.join(directory, f)))

data = []
img_size = 124
mask_classes = ['face_with_mask', 'mask_surgical', 'mask_colorful']
non_mask_classes = ["face_no_mask"]
labels = {'face_with_mask': 0, 'face_without_mask': 1}

for index, row in df.iterrows():
    annotations = jsonfiles[index].get("Annotations")
    for annotation in annotations:
        if annotation["classname"] in mask_classes or annotation["classname"] in non_mask_classes:
            # Ensure the bounding box coordinates are valid
            if len(annotation["BoundingBox"]) == 4:
                x, y, w, h = annotation["BoundingBox"]
                img = cv2.imread(os.path.join(image_directory, row["name"]), 1)

                # Check if the image is not empty
                if img is not None:
                    img = img[y:h, x:w]

                    # Additional check for empty image after cropping
                    if not np.all(np.isnan(img)):
                        img = cv2.resize(img, (img_size, img_size))
                        if annotation["classname"] in mask_classes:
                            data.append([img, labels['face_with_mask']])
                        elif annotation["classname"] in non_mask_classes:
                            data.append([img, labels['face_without_mask']])
                    else:
                        print(f"Error: Empty image after cropping for {row['name']}")
                        print(f"Bounding Box Coordinates: {x}, {y}, {w}, {h}")
            else:
                print(f"Error: Invalid bounding box coordinates for {row['name']}")
                print(f"Bounding Box Coordinates: {annotation['BoundingBox']}")

random.shuffle(data)

p = []
for face in data:
    if face[1] == labels['face_with_mask']:
        p.append("Mask")
    else:
        p.append("No Mask")

# Create a count plot using Seaborn
sns.set(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.countplot(x=p, palette="Set1")
plt.title("Mask vs No Mask")
plt.xlabel("Category")
plt.ylabel("Count")
plt.show()

p = []
X = []
Y = []

for features, label in data:
    X.append(features)
    Y.append(label)

X = np.array(X) / 255.0
X = X.reshape(-1, 124, 124, 3)
Y = np.array(Y)


Error: Empty image after cropping for 5392.jpg
Bounding Box Coordinates: 764, 138, 794, 169
Error: Empty image after cropping for 4287.png
Bounding Box Coordinates: 717, 118, 762, 180
Error: Empty image after cropping for 4287.png
Bounding Box Coordinates: 718, 150, 754, 175
Error: Empty image after cropping for 4070.png
Bounding Box Coordinates: 768, 287, 845, 396
Error: Empty image after cropping for 4070.png
Bounding Box Coordinates: 806, 306, 954, 462
Error: Empty image after cropping for 4070.png
Bounding Box Coordinates: 774, 346, 828, 396
Error: Empty image after cropping for 4070.png
Bounding Box Coordinates: 808, 380, 916, 457
Error: Empty image after cropping for 3673.png
Bounding Box Coordinates: 860, 173, 915, 252
Error: Empty image after cropping for 3673.png
Bounding Box Coordinates: 863, 210, 910, 249
Error: Empty image after cropping for 3673.png
Bounding Box Coordinates: 715, 170, 770, 232
Error: Empty image after cropping for 3673.png
Bounding Box Coordinates: 721, 20

In [8]:
img.size

0

In [ ]:
X = []
Y = []

for features, label in data:
    X.append(features)
    Y.append(label)

X = np.array(X) / 255.0
X = X.reshape(-1, 124, 124, 3)
Y = np.array(Y)

In [ ]:
# Check if there are GPUs available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

# Check which GPU is being used (if any)
print("Available GPUs:")
for device in tf.config.experimental.list_physical_devices('GPU'):
    print(device.name)

In [ ]:
"""model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same", activation='relu', input_shape=(124, 124, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

xtrain, xval, ytrain, yval = train_test_split(X, Y, train_size=0.8, random_state=0)

datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)

datagen.fit(xtrain)

history = model.fit(
    datagen.flow(xtrain, ytrain, batch_size=32),
    steps_per_epoch=xtrain.shape[0] // 32,
    epochs=25,
    verbose=1,
    validation_data=(xval, yval)
)"""

model.save("my_model.h5")

In [ ]:
"""model.save("my_model.h5")"""

In [ ]:
#### To load a saved model, you can use the following code
loaded_model = tf.keras.models.load_model("my_model.h5")

In [ ]:
num_test_images = 6  # Change this to the number of random image names you want to select
test_images = df['name'].sample(n=num_test_images, random_state=42).tolist()

gamma = 2.2 # the gamma value for adjusting image brightness
conf_threshold = 0.2  # the confidence threshold
nms_threshold = 0.9  # the NMS threshold

fig = plt.figure(figsize=(14, 14))
rows = 3
cols = 2
axes = []
assign = {'0': 'Mask', '1': "No Mask"}

for j, im in enumerate(test_images):
    image = cv2.imread(os.path.join(image_directory, im), 1)
    
    # Adjust gamma for better visibility
    image = adjust_gamma(image, gamma=gamma)
    
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (256, 192)), 1.0, (124, 124), (104.0, 177.0, 123.0))
    cvNet.setInput(blob)
    detections = cvNet.forward()

    # Use NMS to filter out overlapping and low-confidence bounding boxes
    indices = cv2.dnn.NMSBoxes(detections[0, 0, :, :4], detections[0, 0, :, 2], conf_threshold, nms_threshold)

    for i in range(len(indices)):
        index = indices[i]  # Get the index from indices
        box = detections[0, 0, index, 3:7] * np.array([w, h, w, h])
        confidence = detections[0, 0, index, 2]

        # Check if confidence is above threshold and box coordinates are valid
        if confidence > conf_threshold:
            (startX, startY, endX, endY) = box.astype("int")
            frame = image[startY:endY, startX:endX]

            im = cv2.resize(frame, (img_size, img_size))
            im = np.array(im) / 255.0
            im = im.reshape(1, 124, 124, 3)

            try:
                result = model.predict(im)
            except:
                result = loaded_model.predict(im)

            if result >= 0.5:
                label_Y = 1
            else:
                label_Y = 0

            cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
            cv2.putText(image, assign[str(label_Y)], (startX, startY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (36, 255, 12), 2)

    axes.append(fig.add_subplot(rows, cols, j + 1))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
def test_pretrained_model(model, image_directory, df_test, img_size=124, gamma=2.0, conf_threshold=0.2, nms_threshold=0.1, prediction_threshold=0.5, num_test_images=6):
    correct_predictions = 0
    total_images = len(df_test)

    # Assuming 'df' contains the necessary information
    # Replace this with your actual column names if they are different
    X = df[['name', 'x1', 'x2', 'y1', 'y2', 'classname']]
    Y = df['label']  # Replace 'label' with your actual column name for class labels

    # Split the data into training and validation sets
    xtrain, xval, ytrain, yval = train_test_split(X, Y, train_size=0.8, random_state=0)

    # Take a random subset of validation data for testing
    random_subset_indices = random.sample(range(len(xval)), num_test_images)
    xtest_subset = xval.iloc[random_subset_indices]
    ytest_subset = yval.iloc[random_subset_indices]

    for index, row in xtest_subset.iterrows():
        im = row['name']
        ground_truth_label = ytest_subset.loc[index]

        # Load the image
        image_path = os.path.join(image_directory, im)
        image = cv2.imread(image_path, 1)

        if image is None:
            print(f"Error loading image: {image_path}")
            continue

        # Adjust gamma for better visibility
        image = adjust_gamma(image, gamma=gamma)

        (h, w) = image.shape[:2]
        # Use the same dimensions for detection and resizing
        blob = cv2.dnn.blobFromImage(image, 1.0, (img_size, img_size), (104.0, 177.0, 123.0))
        cvNet.setInput(blob)
        detections = cvNet.forward()

        # Use NMS to filter out overlapping and low-confidence bounding boxes
        indices = cv2.dnn.NMSBoxes(detections[0, 0, :, :4], detections[0, 0, :, 2], conf_threshold, nms_threshold)

        if len(indices) > 0:
            index = indices[0]  # Get the first index from indices
            box = detections[0, 0, index, 3:7] * np.array([w, h, w, h])
            confidence = detections[0, 0, index, 2]

            # Check if confidence is above threshold and box coordinates are valid
            if confidence > conf_threshold:
                (startX, startY, endX, endY) = box.astype("int")

                # Check if the bounding box coordinates are valid
                if startX < endX and startY < endY:
                    frame = image[startY:endY, startX:endX]

                    # Check if the frame is not empty
                    if frame.size == 0:
                        print(f"Error: Empty frame for image {im}")
                        continue

                    im = cv2.resize(frame, (img_size, img_size))
                    im = np.array(im) / 255.0
                    im = im.reshape(1, img_size, img_size, 3)

                    try:
                        result = model.predict(im)
                    except:
                        result = model.predict(im)

                    if result >= prediction_threshold:
                        label_Y = 1
                    else:
                        label_Y = 0

                    # Check if the predicted label matches the ground truth
                    if label_Y == labels[ground_truth_label]:
                        correct_predictions += 1
                    else:
                        print(f"Mismatch: Predicted {label_Y}, Ground Truth {labels[ground_truth_label]} for image {im}")

    accuracy = correct_predictions / num_test_images
    return accuracy

# Test the pre-trained model and get accuracy
accuracy = test_pretrained_model(loaded_model, image_directory, df_test)
print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
df['classname'] = df['classname'].apply(lambda x: 'face_with_mask' if x in ['face_with_mask', 'mask_surgical', 'mask_colorful'] else 'face_without_mask')

# Verify the unique values in the 'classname' column after conversion
print(df['classname'].unique())